# Stats Practice

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
import sys
print(sys.version)

3.7.7 (tags/v3.7.7:d7c567b08f, Mar 10 2020, 10:41:24) [MSC v.1900 64 bit (AMD64)]


### [예제 1] 지지율 여론 조사
선거 후보에 대해 지지율 조사를 하였다.  
조사 샘플 수는 1,000명이었고, A 후보 507명, B 후보 298명, 기타 195명이었다.  
두 후보의 지지율에 대해 95% 신뢰 구간을 추정하시오.

[용어]
* 유의수준 : $\alpha$
* 신뢰수준 : $(1-\alpha)100\%$
* 표준편차 : $\sigma$
* 표준오차 : $\sigma\sqrt{n}$
* 표본오차 (허용오차, 오차범위) : $z*\sigma\sqrt{n}$

In [8]:
# 유의수준
alpha = 0.05 

# 한 모집단에서 여러 개의 모비율을 추정할 때, 동일한 최대허용 표본오차를 적용한다. (표본오차는 p_hat이 0.5일 때 최대가 된다.)
n_A = 507
n_B = 298
n_X = 195
n_Tot = n_A + n_B + n_X

# 표본비율 (Sampling Ratio) : p_hat = x/n
p_hat_A = n_A / n_Tot 
p_hat_B = n_B / n_Tot

# 확률변수(Random Variable)과 z값 (z-value, z-score)
rv_norm = stats.norm(0,1) # 표준 정규분포(Z=norm(0,1)) 사용
z_val = rv_norm.ppf(1-alpha/2)
err_max = z_val*np.sqrt(0.5*0.5/n_Tot)

p_hat_A_l = p_hat_A - err_max
p_hat_A_r = p_hat_A + err_max
print(round(p_hat_A_l,2), ' <= p_hat_A <= ', round(p_hat_A_r,2), '[', round(p_hat_A*100,0), '+/-', round(err_max*100,0), ']')

p_hat_B_l = p_hat_B - err_max
p_hat_B_r = p_hat_B + err_max
print(round(p_hat_B_l,2), ' <= p_hat_B <= ', round(p_hat_B_r,2), '[', round(p_hat_B*100,0), '+/-', round(err_max*100,0), ']')

0.48  <= p_hat_A <=  0.54 [ 51.0 +/- 3.0 ]
0.27  <= p_hat_B <=  0.33 [ 30.0 +/- 3.0 ]
